In [ ]:
import openai
from dotenv import load_dotenv
import os
import requests
import subprocess
import json
import asyncio
from api_request_parallel_processor import process_api_requests_from_file


load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

In [4]:
temperature = 0.7
prompt_stem = "Hi, what is the capital of "
ten_countries = ['France', 'Germany', 'Italy', 'Spain', 'United Kingdom', 'United States', 'Canada', 'Mexico', 'Brazil', 'Argentina']
prompts = [prompt_stem + country + '?' for country in ten_countries]
data_list = []
for p in prompts:
    data = {"model": "gpt-4-0613", "messages": [{"role": "user", "content": p}],"temperature": temperature}
    data_list.append(data)

# create jsonl file 
with open('data.jsonl', 'w') as outfile:
    for entry in data_list:
        json.dump(entry, outfile)
        outfile.write('\n')



In [29]:
import subprocess

# Define the command to call the script
command = [
    "python", "api_request_parallel_processor.py",
    "--requests_filepath", "data.jsonl",
    "--save_filepath", "data_responses.jsonl",
    "--request_url", "https://api.openai.com/v1/chat/completions",
    "--max_requests_per_minute", "200",
    "--max_tokens_per_minute", "4000",
    "--token_encoding_name", "cl100k_base",
    "--max_attempts", "5",
    "--logging_level", "10"
]

# Launch the Windows shell command
subprocess.run(command, shell=True)

CompletedProcess(args=['python', 'api_request_parallel_processor.py', '--requests_filepath', 'data.jsonl', '--save_filepath', 'data_responses.jsonl', '--request_url', 'https://api.openai.com/v1/chat/completions', '--max_requests_per_minute', '200', '--max_tokens_per_minute', '4000', '--token_encoding_name', 'cl100k_base', '--max_attempts', '5', '--logging_level', '10'], returncode=0)

In [5]:
import subprocess

# Define the command to call the script
command = [
    "python", "api_request_parallel_processor.py",
    "--requests_filepath", "data.jsonl",
    "--save_filepath", "data_responses.jsonl",
    "--request_url", "https://api.openai.com/v1/chat/completions",
    "--max_requests_per_minute", "200",
    "--max_tokens_per_minute", "4000",
    "--token_encoding_name", "cl100k_base",
    "--max_attempts", "5",
    "--logging_level", "10"
]

# Launch the Windows shell command and get the output
output = subprocess.check_output(command, shell=True)

# Decode the bytes to string and print the output
print(output.decode("utf-8"))

In [7]:
import os
import json
import subprocess

def parallel_gpt4_request(messages, model, temperature, api_key, max_requests_per_minute = 200, max_tokens_per_minute = 4000, max_attempts = 5, logging_level = 10):
    """
    Make requests in system_message, user_message pairs. 
    format messages as a list of tuples, [(system_message_1, user_message_1), (system_message_2, user_message_2), ...]
    if system message is None, then it is a user message only
    """
    # remove 00temp.jsonl if it exists
    if os.path.exists('00temp.jsonl'):
        os.remove('00temp.jsonl')
    if os.path.exists('00temp_responses.jsonl'):
        os.remove('00temp_responses.jsonl')

    # create jsonl file 
    with open('00temp.jsonl', 'w') as outfile:
        for entry in messages:
            if entry[0] is None:
                data = {"model": model, "messages": [{"role": "user", "content": entry[1]}],"temperature": temperature}
            else:
                data = {"model": model, "messages": [{"role": "system", "content": entry[0]}, {"role": "user", "content": entry[1]}],"temperature": temperature}
            json.dump(data, outfile)
            outfile.write('\n')

    # Define the command to call the script
    command = [
        "python", "api_request_parallel_processor.py",
        "--requests_filepath", "00temp.jsonl",
        "--save_filepath", "00temp_responses.jsonl",
        "--request_url", "https://api.openai.com/v1/chat/completions",
        "--api_key", api_key,
        "--max_requests_per_minute", str(max_requests_per_minute),
        "--max_tokens_per_minute", str(max_tokens_per_minute),
        "--token_encoding_name", "cl100k_base",
        "--max_attempts", str(max_attempts),
        "--logging_level", str(logging_level)
    ]

    # Launch the Windows shell command and get the output
    output = subprocess.check_output(command, shell=True)

    # Decode the bytes to string and print the output
    print(output.decode("utf-8"))

    # read in responses
    with open('00temp_responses.jsonl', 'r') as f:
        responses = [json.loads(line) for line in f]
    request_messages_dict  = [response[0]['messages'] for response in responses]
    response_messages = [response[1]['choices'][0]['message']['content'] for response in responses]

    request_messages = []
    for rm in request_messages_dict:
        if len(rm) == 1: #contains only user message
            request_messages.append((None, rm[0]['content']))
        elif len(rm) == 2: #contains system and user message
            request_messages.append((rm[0]['content'], rm[1]['content']))
        else:
            raise Exception('Request message has no body')
    

    # remove 00temp.jsonl if it exists
    # if os.path.exists('00temp.jsonl'):
        # os.remove('00temp.jsonl')
    if os.path.exists('00temp_responses.jsonl'):
        os.remove('00temp_responses.jsonl')
    
    return request_messages, response_messages

prompts = [(None, "What is the capital of France?"), ("You are Donald Trump", "Who is the best president")]

request_messages, response_messages = parallel_gpt4_request(prompts, "gpt-4-0613", 0, os.getenv("OPENAI_API_KEY"), max_requests_per_minute = 200, max_tokens_per_minute = 4000, max_attempts = 5, logging_level = 10)


In [6]:
response_messages

['The capital of France is Paris.',
 'As Donald Trump, I would say that I am the best president. I believe I have done a great job in leading the country, creating jobs, and making America great again.']

In [56]:
request_messages
response_messages

[(None, 'What is the capital of France?'),
 ('You are Donald Trump', 'Who is the best president')]

In [57]:
response_messages

['The capital of France is Paris.',
 'Many people have different opinions on who the best president is. However, as Donald Trump, I would say that I am the best president.']

[[{'role': 'user', 'content': 'What is the capital of France?'}],
 [{'role': 'system', 'content': 'You are Donald Trump'},
  {'role': 'user', 'content': 'Who is the best president'}]]

In [51]:
request_messages

[[{'role': 'user', 'content': 'Hi, what is the capital of Italy?'}],
 [{'role': 'user', 'content': 'Hi, what is the capital of France?'}],
 [{'role': 'user', 'content': 'Hi, what is the capital of Mexico?'}],
 [{'role': 'user', 'content': 'Hi, what is the capital of United Kingdom?'}],
 [{'role': 'user', 'content': 'Hi, what is the capital of Brazil?'}],
 [{'role': 'user', 'content': 'Hi, what is the capital of Spain?'}],
 [{'role': 'user', 'content': 'Hi, what is the capital of Argentina?'}],
 [{'role': 'user', 'content': 'Hi, what is the capital of Germany?'}],
 [{'role': 'user', 'content': 'Hi, what is the capital of Canada?'}],
 [{'role': 'user', 'content': 'Hi, what is the capital of United States?'}],
 [{'role': 'user', 'content': 'Hi, what is the capital of Argentina?'}],
 [{'role': 'user', 'content': 'Hi, what is the capital of Brazil?'}],
 [{'role': 'user', 'content': 'Hi, what is the capital of France?'}],
 [{'role': 'user', 'content': 'Hi, what is the capital of Mexico?'}],


In [49]:
request_messages
response_messages

['The capital of Italy is Rome.',
 'The capital of France is Paris.',
 'The capital of Mexico is Mexico City.',
 'The capital of the United Kingdom is London.',
 'The capital of Brazil is Brasília.',
 'The capital of Spain is Madrid.',
 'The capital of Argentina is Buenos Aires.',
 'The capital of Germany is Berlin.',
 'The capital of Canada is Ottawa.',
 'The capital of the United States is Washington, D.C.',
 'The capital of Argentina is Buenos Aires.',
 'The capital of Brazil is Brasília.',
 'The capital of France is Paris.',
 'The capital of Mexico is Mexico City.',
 'The capital of Italy is Rome.',
 'The capital of Canada is Ottawa.',
 'The capital of the United Kingdom is London.',
 'The capital of the United States is Washington, D.C.',
 'The capital of Germany is Berlin.',
 'The capital of Spain is Madrid.']

In [ ]:
def parallel_gpt4_request_from_file(requests_filepath, save_filepath, max_requests_per_minute = 200, max_tokens_per_minute = 4000, max_attempts = 5, logging_level = 10):
    # remove save_filepath if exists
    if os.path.exists(save_filepath):
        os.remove(save_filepath)
    
    # Define the command to call the script
    command = [
        "python", "api_request_parallel_processor.py",
        "--requests_filepath", requests_filepath,
        "--save_filepath", save_filepath,
        "--request_url", "https://api.openai.com/v1/chat/completions",
        "--max_requests_per_minute", str(max_requests_per_minute),
        "--max_tokens_per_minute", str(max_tokens_per_minute),
        "--token_encoding_name", "cl100k_base",
        "--max_attempts", str(max_attempts),
        "--logging_level", str(logging_level)
    ]

    # Launch the Windows shell command and get the output
    subprocess.run(command, shell=True)

    # read jsonl out and parse as list of dicts, overwrite if exists
    with open(save_filepath) as f:
        lines = f.readlines()
    responses = [json.loads(line) for line in lines]
    request_messages  = [response[0]['messages'] for response in responses]
    response_messages = [response[1]['choices'][0]['message']['content'] for response in responses]
    return responses, request_messages, response_messages 

responses, request_messages, response_messages = parallel_gpt4_request_from_file("data.jsonl", "out.jsonl", 200, 4000, 5, 10)

ValueError: not enough values to unpack (expected 3, got 2)

'['

In [ ]:
import pandas as pd 

data = pd.read_csv('data\APS_dev_data\personalityforum\mbti_1.csv')

import textwrap

for post in data['posts'].iloc[:20]:
    print('\n'.join(textwrap.wrap(post, width=70)))
    print('\n-----------------\n')
    break

'http://www.youtube.com/watch?v=qsXHcwe3krw|||http://41.media.tumblr.c
om/tumblr_lfouy03PMA1qa1rooo1_500.jpg|||enfp and intj moments
https://www.youtube.com/watch?v=iz7lE1g4XM4  sportscenter not top ten
plays  https://www.youtube.com/watch?v=uCdfze1etec  pranks|||What has
been the most life-changing experience in your
life?|||http://www.youtube.com/watch?v=vXZeYwwRDw8
http://www.youtube.com/watch?v=u8ejam5DP3E  On repeat for most of
today.|||May the PerC Experience immerse you.|||The last thing my INFJ
friend posted on his facebook before committing suicide the next day.
Rest in peace~   http://vimeo.com/22842206|||Hello ENFJ7. Sorry to
hear of your distress. It's only natural for a relationship to not be
perfection all the time in every moment of existence. Try to figure
the hard times as times of growth, as...|||84389  84390
http://wallpaperpassion.com/upload/23700/friendship-boy-and-girl-
wallpaper.jpg  http://assets.dornob.com/wp-
content/uploads/2010/04/round-home-design.jpg ...||

In [ ]:
import pandas as pd 

data = pd.read_csv('data\APS_dev_data\personalityforum\mbti_1.csv')

import textwrap

for post in data['posts'].iloc[:20]:
    print('\n'.join(textwrap.wrap(post, width=70)))
    print('\n-----------------\n')
    break

'http://www.youtube.com/watch?v=qsXHcwe3krw|||http://41.media.tumblr.c
om/tumblr_lfouy03PMA1qa1rooo1_500.jpg|||enfp and intj moments
https://www.youtube.com/watch?v=iz7lE1g4XM4  sportscenter not top ten
plays  https://www.youtube.com/watch?v=uCdfze1etec  pranks|||What has
been the most life-changing experience in your
life?|||http://www.youtube.com/watch?v=vXZeYwwRDw8
http://www.youtube.com/watch?v=u8ejam5DP3E  On repeat for most of
today.|||May the PerC Experience immerse you.|||The last thing my INFJ
friend posted on his facebook before committing suicide the next day.
Rest in peace~   http://vimeo.com/22842206|||Hello ENFJ7. Sorry to
hear of your distress. It's only natural for a relationship to not be
perfection all the time in every moment of existence. Try to figure
the hard times as times of growth, as...|||84389  84390
http://wallpaperpassion.com/upload/23700/friendship-boy-and-girl-
wallpaper.jpg  http://assets.dornob.com/wp-
content/uploads/2010/04/round-home-design.jpg ...||

In [30]:
response_messages

['The capital of the United Kingdom is London.',
 'The capital of Mexico is Mexico City.',
 'The capital of Italy is Rome.',
 'The capital of Spain is Madrid.',
 'The capital of Canada is Ottawa.',
 'The capital of Brazil is Brasília.',
 'The capital of Germany is Berlin.',
 'The capital of the United States is Washington, D.C.',
 'The capital of Argentina is Buenos Aires.',
 'The capital of France is Paris.']

In [28]:
responses

(['[{"model": "gpt-4-0613", "messages": [{"role": "user", "content": "Hi, what is the capital of Canada?"}], "temperature": 0}, {"id": "chatcmpl-84lapo27hpAgY3Ao1KFDg4iFlrXln", "object": "chat.completion", "created": 1696146951, "model": "gpt-4-0613", "choices": [{"index": 0, "message": {"role": "assistant", "content": "The capital of Canada is Ottawa."}, "finish_reason": "stop"}], "usage": {"prompt_tokens": 16, "completion_tokens": 7, "total_tokens": 23}}]\n',
  '[{"model": "gpt-4-0613", "messages": [{"role": "user", "content": "Hi, what is the capital of Mexico?"}], "temperature": 0}, {"id": "chatcmpl-84laprFIzRJKgWjoExGjShrCA5PGo", "object": "chat.completion", "created": 1696146951, "model": "gpt-4-0613", "choices": [{"index": 0, "message": {"role": "assistant", "content": "The capital of Mexico is Mexico City."}, "finish_reason": "stop"}], "usage": {"prompt_tokens": 16, "completion_tokens": 8, "total_tokens": 24}}]\n',
  '[{"model": "gpt-4-0613", "messages": [{"role": "user", "cont

In [26]:
responses

NameError: name 'responses' is not defined

In [23]:
len(out)

40

In [22]:
messages = out[0][1]['choices'][0]['message']['content']
[response[1]['choices'][0]['message']['content'] for response in out]

['The capital of France is Paris.',
 'The capital of Argentina is Buenos Aires.',
 'The capital of Canada is Ottawa.',
 'The capital of Germany is Berlin.',
 'The capital of Spain is Madrid.',
 'The capital of Italy is Rome.',
 'The capital of the United Kingdom is London.',
 'The capital of Mexico is Mexico City.',
 'The capital of Brazil is Brasília.',
 'The capital of the United States is Washington, D.C.',
 'The capital of Germany is Berlin.',
 'The capital of Mexico is Mexico City.',
 'The capital of Italy is Rome.',
 'The capital of the United Kingdom is London.',
 'The capital of France is Paris.',
 'The capital of Canada is Ottawa.',
 'The capital of Brazil is Brasília.',
 'The capital of Argentina is Buenos Aires.',
 'The capital of Spain is Madrid.',
 'The capital of the United States is Washington, D.C.',
 'The capital of France is Paris.',
 'The capital of Argentina is Buenos Aires.',
 'The capital of the United Kingdom is London.',
 'The capital of Italy is Rome.',
 'The c

In [12]:
out

[{'model': 'gpt-4-0613',
  'messages': [{'role': 'user',
    'content': 'Hi, what is the capital of France?'}],
  'temperature': 0},
 {'model': 'gpt-4-0613',
  'messages': [{'role': 'user',
    'content': 'Hi, what is the capital of Argentina?'}],
  'temperature': 0},
 {'model': 'gpt-4-0613',
  'messages': [{'role': 'user',
    'content': 'Hi, what is the capital of Canada?'}],
  'temperature': 0},
 {'model': 'gpt-4-0613',
  'messages': [{'role': 'user',
    'content': 'Hi, what is the capital of Germany?'}],
  'temperature': 0},
 {'model': 'gpt-4-0613',
  'messages': [{'role': 'user',
    'content': 'Hi, what is the capital of Spain?'}],
  'temperature': 0},
 {'model': 'gpt-4-0613',
  'messages': [{'role': 'user',
    'content': 'Hi, what is the capital of Italy?'}],
  'temperature': 0},
 {'model': 'gpt-4-0613',
  'messages': [{'role': 'user',
    'content': 'Hi, what is the capital of United Kingdom?'}],
  'temperature': 0},
 {'model': 'gpt-4-0613',
  'messages': [{'role': 'user',
  

In [13]:
out[0]

{'model': 'gpt-4-0613',
 'messages': [{'role': 'user',
   'content': 'Hi, what is the capital of France?'}],
 'temperature': 0}

In [43]:
output

b''

In [41]:
# Prepare headers
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {os.getenv('OPENAI_API_KEY')}",
}

# Prepare data
data = {
    "model": "gpt-4-0613",
    "messages": [{
        "role": "user",
        "content": "Hi, what is the capital of France?"
    }],
    "temperature": 0,
}

# do it 10 times
for i in range(10):
    # Send a POST request
    response = requests.post(
        url="https://api.openai.com/v1/chat/completions",
        headers=headers,
        data=json.dumps(data),
    )

    # Check the response
    if response.status_code == 200:
        r = response.json()
        print(r['choices'][0]['message']['content'])
        
    else:
        print(f"Failed to get response, status code: {response.status_code}")


The capital of France is Paris.
The capital of France is Paris.
The capital of France is Paris.
The capital of France is Paris.
The capital of France is Paris.
The capital of France is Paris.
The capital of France is Paris.
The capital of France is Paris.
The capital of France is Paris.
The capital of France is Paris.


In [40]:
# response = response.json()
response['choices'][0]['message']['content']	

'The capital of France is Paris.'